# Dependencies

In [1]:
# import libraries
import requests
import time
from datetime import datetime
import json
import sys
import pandas as pd
pd.set_option('display.max_columns', 500)  #allows display of all columns in dfs

import numpy as np
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine

In [2]:
# mysql login info
sys.path.insert(0, '../../Key')
from mysql_secret import dbuser, dbpass, dbhost, dbname
engine = create_engine(f'mysql://{dbuser}:{dbpass}@{dbhost}/{dbname}?charset=utf8')

# Scrape IPOs
Retrieve IPO stock information from Nasdaq

In [9]:
current_year_month = datetime.today().strftime('%Y-%m')
current_year_month

'2021-01'

In [27]:
# scrape nasdaq, Ex: https://api.nasdaq.com/api/ipo/calendar?date=2020-08
# note, had to create headers due to time out, solution found here: https://stackoverflow.com/questions/46862719/pythons-requests-library-timing-out-but-getting-the-response-from-the-browser
url = f'https://api.nasdaq.com/api/ipo/calendar?date=2020-12'
headers = {"User-Agent":"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_5) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/12.1.1 Safari/605.1.15","Accept-Language": "en-gb","Accept-Encoding":"br, gzip, deflate","Accept":"test/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8","Referer":"http://www.google.com/"}

response = requests.get(url, headers=headers)
data = response.text
data = json.loads(data)
print(url)

https://api.nasdaq.com/api/ipo/calendar?date=2020-12


In [28]:
# create list of scraped dataframes to concatenate
scraped_ipo_dfs = []

In [29]:
# gets priced IPOs for each record on nasdaq (values) = [(expression) for (value) in (collection)]
priced_ipos = data["data"]["priced"]["rows"]

# if there are priced IPOs create dataframe
if priced_ipos:
    
    symbols = [priced_ipos[x]["proposedTickerSymbol"] for x in range(len(priced_ipos))]
    company = [priced_ipos[x]["companyName"] for x in range(len(priced_ipos))]
    exchange = [priced_ipos[x]["proposedExchange"] for x in range(len(priced_ipos))]
    proposed_share_price = [priced_ipos[x]["proposedSharePrice"] for x in range(len(priced_ipos))]
    shares_offered = [priced_ipos[x]["sharesOffered"].replace(",", '') for x in range(len(priced_ipos))]
    priced_date = [priced_ipos[x]["pricedDate"] for x in range(len(priced_ipos))]
    dollar_val_shares = [priced_ipos[x]["dollarValueOfSharesOffered"].replace(",", '').replace("$",'') for x in range(len(priced_ipos))]

    # dataframe with stock info
    nasdaq_priced_df = pd.DataFrame({"symbol" : symbols,
                                       "company" : company,
                                       "exchange" : exchange, 
                                       "proposed_share_price" : proposed_share_price,
                                       "shares_offered" : shares_offered,
                                       "priced_date" : priced_date,
                                       "dollar_val_shares" : dollar_val_shares,
                                       "deal_status" : "priced"
                                       })
    scraped_ipo_dfs.append(nasdaq_priced_df)
    print(f"{len(priced_ipos)} priced IPOs")
else:
    print("no priced IPOs")

74 priced IPOs


In [30]:
# gets lists for each record on nasdaq (values) = [(expression) for (value) in (collection)]

upcoming_ipos = data["data"]["upcoming"]["upcomingTable"]["rows"]

if upcoming_ipos:
    symbols = [upcoming_ipos[x]["proposedTickerSymbol"] for x in range(len(upcoming_ipos))]
    company = [upcoming_ipos[x]["companyName"] for x in range(len(upcoming_ipos))]
    exchange = [upcoming_ipos[x]["proposedExchange"] for x in range(len(upcoming_ipos))]
    proposed_share_price = [upcoming_ipos[x]["proposedSharePrice"] for x in range(len(upcoming_ipos))]
    shares_offered = [upcoming_ipos[x]["sharesOffered"].replace(",", '') for x in range(len(upcoming_ipos))]
    priced_date = [upcoming_ipos[x]["expectedPriceDate"] for x in range(len(upcoming_ipos))]
    dollar_val_shares = [upcoming_ipos[x]["dollarValueOfSharesOffered"].replace(",", '').replace("$",'') for x in range(len(upcoming_ipos))]

    # dataframe with stock info
    nasdaq_upcoming_df = pd.DataFrame({"symbol" : symbols,
                                       "company" : company,
                                       "exchange" : exchange, 
                                       "proposed_share_price" : proposed_share_price,
                                       "shares_offered" : shares_offered,
                                       "priced_date" : priced_date,
                                       "dollar_val_shares" : dollar_val_shares,
                                       "deal_status" : "expected"
                                       })
    scraped_ipo_dfs.append(nasdaq_upcoming_df)
    print(f"{len(upcoming_ipos)} upcoming IPOs")
else: 
    print("no upcoming IPOs")

1 upcoming IPOs


In [31]:
# combine IPO dataframes
ipo_df = pd.concat(scraped_ipo_dfs, ignore_index=True, sort=False)

# change column datatypes
ipo_df[['shares_offered', 'dollar_val_shares']] = ipo_df[['shares_offered', 'dollar_val_shares']].apply(pd.to_numeric)
ipo_df['priced_date'] = pd.to_datetime(ipo_df['priced_date'], format="%m/%d/%Y")
ipo_df = ipo_df.sort_values(by='deal_status', ascending=False).reset_index(drop=True)
ipo_df = ipo_df.drop_duplicates(subset="symbol", keep="first")
print(ipo_df.dtypes)
ipo_df = ipo_df.dropna()
ipo_df

symbol                          object
company                         object
exchange                        object
proposed_share_price            object
shares_offered                   int64
priced_date             datetime64[ns]
dollar_val_shares              float64
deal_status                     object
dtype: object


,symbol,company,exchange,proposed_share_price,shares_offered,priced_date,dollar_val_shares,deal_status
0,MRM,Medirom Healthcare Technologies Inc.,NASDAQ Capital,15.00,800000,2020-12-29,1.200000e+07,priced
1,MUDSU,Mudrick Capital Acquisition Corp. II,NASDAQ Capital,10.00,27500000,2020-12-08,2.750000e+08,priced
2,PUBM,"PubMatic, Inc.",NASDAQ Global,20.00,5900000,2020-12-09,1.180000e+08,priced
3,DASH,DoorDash Inc,NYSE,102.00,33000000,2020-12-09,3.366000e+09,priced
4,ALTUU,Altitude Acquisition Corp.,NASDAQ Capital,10.00,26100000,2020-12-09,2.610000e+08,priced
...,...,...,...,...,...,...,...,...
70,WNW,Wunong Net Technology Co Ltd,NASDAQ Capital,5.00,5999910,2020-12-15,2.999955e+07,priced
71,SCPS,Scopus Biopharma Inc.,NASDAQ Global,5.50,500000,2020-12-16,2.750000e+06,priced
72,WISH,ContextLogic Inc.,NASDAQ Global Select,24.00,46000000,2020-12-16,1.104000e+09,priced
73,UPST,"Upstart Holdings, Inc.",NASDAQ Global Select,20.00,12015690,2020-12-16,2.403138e+08,priced


# New IPOs to Database

In [37]:
# Bring in ipo table
engine = create_engine(f'mysql://{dbuser}:{dbpass}@{dbhost}/{dbname}?charset=utf8')
connection = engine.connect()
sql_ipo_df = pd.read_sql("SELECT * FROM stocks", connection)
sql_ipo_df.head()

,symbol,company,exchange,proposed_share_price,shares_offered,priced_date,dollar_val_shares,deal_status,proposed_cap_classifiction
0,AAAU,Goldman Sachs Physical Gold ETF,NYSE Arca,11.74,16286645,2018-08-15,1.912052e+08,priced,Micro-cap
1,AACQU,Artius Acquisition Inc.,NASDAQ Capital,10.00,63000000,2020-07-14,6.300000e+08,priced,Small-cap
2,ABCL,AbCellera Biologics Inc.,NASDAQ Global Select,20.00,24150000,2020-12-11,4.830000e+08,priced,Small-cap
3,ABCM,Abcam PLC,NASDAQ Global Select,17.50,8945218,2020-10-22,1.565413e+08,priced,Micro-cap
4,ABNB,"Airbnb, Inc.",NASDAQ Global Select,68.00,51323531,2020-12-10,3.490000e+09,priced,Mid-cap


In [10]:
new_ipos_df = ipo_df[~ipo_df["symbol"].isin(sql_ipo_df["symbol"])]
new_ipos_df.head()
new_ipos_df

,symbol,company,exchange,proposed_share_price,shares_offered,priced_date,dollar_val_shares,deal_status


In [ ]:
# load data
engine = create_engine(f'mysql://{dbuser}:{dbpass}@{dbhost}/{dbname}?charset=utf8')
new_ipos_df.to_sql('stocks', con=engine, if_exists='append', index=False)

# Update IPOs in Database
Any IPO that is deal status "expected" may have new data, so records should be updated to reflect most recent pull

In [75]:
# query IPOs with expected status in SQL database
sql_expected = sql_ipo_df.query("deal_status == 'expected'")
sql_expected

,symbol,company,exchange,proposed_share_price,shares_offered,priced_date,dollar_val_shares,deal_status,proposed_cap_classifiction
114,BENEU,Benessere Capital Acquisition Corp.,NASDAQ Capital,10.00,10000000,2020-12-29,115000000.0,expected,Micro-cap
666,PHAR,Pharming Group N.V.,NASDAQ Global,2,5,2020-12-15,10887604.2,expected,Nano-cap


In [82]:
# expected stocks in SQL database that are in new IPO pull from nasdaq 
stocks_to_update = ipo_df[ipo_df["symbol"].isin(sql_expected["symbol"])]
stocks_to_update

,symbol,company,exchange,proposed_share_price,shares_offered,priced_date,dollar_val_shares,deal_status
74,PHAR,Pharming Group N.V.,NASDAQ Global,12.10,899802,2020-12-22,10887604.2,expected


In [83]:
# create engine
engine = create_engine(f'mysql://{dbuser}:{dbpass}@{dbhost}/{dbname}?charset=utf8')

# Declare a Base using `automap_base()`
Base = automap_base()

# Use the Base class to reflect the database tables
Base.prepare(engine, reflect=True)

# Print all of the classes mapped to the Base
Base.classes.keys()

['company_info', 'stocks', 'market_cap', 'performance', 'stock_issues']

In [84]:
# Assign the ipo class to a variable called `IPO`
Stocks = Base.classes.stocks

# Create a session
session = Session(engine)

In [85]:
# Use the session to query ipo table and display the first 5 symbols
for r in session.query(Stocks.symbol, Stocks.company).limit(5).all():
    print(r)

('AAAU', 'Goldman Sachs Physical Gold ETF')
('AACQU', 'Artius Acquisition Inc.')
('ABCL', 'AbCellera Biologics Inc.')
('ABCM', 'Abcam PLC')
('ABNB', 'Airbnb, Inc.')


In [87]:
for row in stocks_to_update.itertuples():

    # Get the record we want to change
    stock = session.query(Stocks).filter(Stocks.symbol==row.symbol).first()

    # Change the record
    stock.proposed_share_price = stocks_to_update.loc[stocks_to_update["symbol"] == row.symbol, 'proposed_share_price'].iloc[0]
    stock.shares_offered = stocks_to_update.loc[stocks_to_update["symbol"] == row.symbol, 'shares_offered'].iloc[0]
    stock.priced_date = stocks_to_update.loc[stocks_to_update["symbol"] == row.symbol, 'priced_date'].iloc[0]
    stock.dollar_val_shares = stocks_to_update.loc[stocks_to_update["symbol"] == row.symbol, 'dollar_val_shares'].iloc[0]
    stock.deal_status = stocks_to_update.loc[stocks_to_update["symbol"] == row.symbol, 'deal_status'].iloc[0]

# Update the database
session.commit()

# Stock Performance
To find these URLs go to a stock tickers page on Yahoo finance. When clicking on a tab open inspector > network > XHR > headers.  URL will display. 

General URL:  
https://query1.finance.yahoo.com/v8/finance/chart/AAPL?formatted=true&crumb=T18HKACbWPn&lang=en-US&region=US&events=div%7Csplit&includeAdjustedClose=true&interval=1d&range=2y&corsDomain=finance.yahoo.com

URL with Unix Date Selection (period1=xxxxxxx&period2=xxxxxxxxxx):
https://query2.finance.yahoo.com/v8/finance/chart/AAPL?formatted=true&crumb=T18HKACbWPn&lang=en-US&region=US&includeAdjustedClose=true&interval=1d&period1=1546300800&period2=1608076800&events=div%7Csplit&corsDomain=finance.yahoo.com

In [12]:
# get unix time range for web request
start_unixtime = 1514903400  #Jan 2, 2018

current_date = datetime.now()
print(current_date)

earliest_date_time = current_date.replace(hour=0, minute=0, second=0, microsecond=0)
print(earliest_date_time)

end_unixtime = time.mktime(earliest_date_time.timetuple())
end_unixtime = int(end_unixtime)
end_unixtime

2021-01-08 19:58:09.234962
2021-01-08 00:00:00


1610085600

In [13]:
today = datetime.today().strftime("%Y-%m-%d")
print(today)

2021-01-08


In [14]:
# query stocks from SQL
connection = engine.connect()
ipo_stocks = pd.read_sql("SELECT symbol, deal_status FROM stocks WHERE deal_status = 'priced'", connection)
ipo_stocks["default_start_unixtime"] = start_unixtime
ipo_stocks["end_unixtime"] = end_unixtime
ipo_stocks

,symbol,deal_status,default_start_unixtime,end_unixtime
0,AAAU,priced,1514903400,1610085600
1,AACQU,priced,1514903400,1610085600
2,ABCL,priced,1514903400,1610085600
3,ABCM,priced,1514903400,1610085600
4,ABNB,priced,1514903400,1610085600
...,...,...,...,...
967,ZM,priced,1514903400,1610085600
968,ZNTEU,priced,1514903400,1610085600
969,ZNTL,priced,1514903400,1610085600
970,ZS,priced,1514903400,1610085600


In [15]:
stock_perf_start_unix = pd.read_sql("SELECT symbol, max(unix_time) AS max_unix_captured  FROM performance  GROUP BY symbol""", connection)
ipo_stocks = ipo_stocks.merge(stock_perf_start_unix, on="symbol", how="outer")
ipo_stocks["max_unix_captured"] = ipo_stocks["max_unix_captured"].fillna(0).astype('int64')
ipo_stocks["max_unix_captured"] = ipo_stocks["max_unix_captured"] + 86400  #add a day to latest date captured
ipo_stocks

,symbol,deal_status,default_start_unixtime,end_unixtime,max_unix_captured
0,AAAU,priced,1514903400,1610085600,1608906600
1,AACQU,priced,1514903400,1610085600,1608906600
2,ABCL,priced,1514903400,1610085600,1608906600
3,ABCM,priced,1514903400,1610085600,1608906600
4,ABNB,priced,1514903400,1610085600,1608906600
...,...,...,...,...,...
967,ZM,priced,1514903400,1610085600,1608906600
968,ZNTEU,priced,1514903400,1610085600,1608906600
969,ZNTL,priced,1514903400,1610085600,1608906600
970,ZS,priced,1514903400,1610085600,1608906600


In [16]:
ipo_stocks["start_unixtime"] = ipo_stocks[["default_start_unixtime", "max_unix_captured"]].max(axis=1).astype('int64') 
ipo_stocks

,symbol,deal_status,default_start_unixtime,end_unixtime,max_unix_captured,start_unixtime
0,AAAU,priced,1514903400,1610085600,1608906600,1608906600
1,AACQU,priced,1514903400,1610085600,1608906600,1608906600
2,ABCL,priced,1514903400,1610085600,1608906600,1608906600
3,ABCM,priced,1514903400,1610085600,1608906600,1608906600
4,ABNB,priced,1514903400,1610085600,1608906600,1608906600
...,...,...,...,...,...,...
967,ZM,priced,1514903400,1610085600,1608906600,1608906600
968,ZNTEU,priced,1514903400,1610085600,1608906600,1608906600
969,ZNTL,priced,1514903400,1610085600,1608906600,1608906600
970,ZS,priced,1514903400,1610085600,1608906600,1608906600


In [17]:
# empty list of dfs
performance_df_list= []

# make requests for performance information for each row
for row in ipo_stocks.itertuples():

    url = f'https://query2.finance.yahoo.com/v8/finance/chart/{row.symbol}?formatted=true&crumb=T18HKACbWPn&lang=en-US&region=US&includeAdjustedClose=true&interval=1d&period1={row.start_unixtime}&period2={row.end_unixtime}&events=div%7Csplit&corsDomain=finance.yahoo.com'
    r = requests.get(url)
    print(f"trying url: {url}")
    if r.ok:
        try: 
            data = r.json()

            # get data
            timestamp = data["chart"]["result"][0]["timestamp"]
            stk_open = data["chart"]["result"][0]["indicators"]["quote"][0]["open"]
            stk_close = data["chart"]["result"][0]["indicators"]["quote"][0]["close"]
            stk_high = data["chart"]["result"][0]["indicators"]["quote"][0]["high"]
            stk_low = data["chart"]["result"][0]["indicators"]["quote"][0]["low"]
            stk_vol = data["chart"]["result"][0]["indicators"]["quote"][0]["volume"]

            #transform into dataframe
            df = pd.DataFrame({"symbol" : row.symbol,
                               "unix_time" : timestamp,
                               "date" : [datetime.fromtimestamp(ts).strftime('%Y-%m-%d') for ts in timestamp],
                               "open" : stk_open, 
                               "close" : stk_close,
                               "high" : stk_high,
                               "low" : stk_low,
                               "volume" : stk_vol
                              })

            df['date'] = pd.to_datetime(df['date'], format="%Y-%m-%d")
            df['date_pulled'] = today

            print(f"{row.symbol} has results")
            performance_df_list.append(df)
        except KeyError:
            continue
    
    else:
        print(f"{row.symbol} NO RESULTS")

    time.sleep(.5)
    

trying url: https://query2.finance.yahoo.com/v8/finance/chart/AAAU?formatted=true&crumb=T18HKACbWPn&lang=en-US&region=US&includeAdjustedClose=true&interval=1d&period1=1608906600&period2=1610085600&events=div%7Csplit&corsDomain=finance.yahoo.com
AAAU has results
trying url: https://query2.finance.yahoo.com/v8/finance/chart/AACQU?formatted=true&crumb=T18HKACbWPn&lang=en-US&region=US&includeAdjustedClose=true&interval=1d&period1=1608906600&period2=1610085600&events=div%7Csplit&corsDomain=finance.yahoo.com
AACQU has results
trying url: https://query2.finance.yahoo.com/v8/finance/chart/ABCL?formatted=true&crumb=T18HKACbWPn&lang=en-US&region=US&includeAdjustedClose=true&interval=1d&period1=1608906600&period2=1610085600&events=div%7Csplit&corsDomain=finance.yahoo.com
ABCL has results
trying url: https://query2.finance.yahoo.com/v8/finance/chart/ABCM?formatted=true&crumb=T18HKACbWPn&lang=en-US&region=US&includeAdjustedClose=true&interval=1d&period1=1608906600&period2=1610085600&events=div%7Csp

trying url: https://query2.finance.yahoo.com/v8/finance/chart/AJAXU?formatted=true&crumb=T18HKACbWPn&lang=en-US&region=US&includeAdjustedClose=true&interval=1d&period1=1514903400&period2=1610085600&events=div%7Csplit&corsDomain=finance.yahoo.com
AJAXU NO RESULTS
trying url: https://query2.finance.yahoo.com/v8/finance/chart/AKICU?formatted=true&crumb=T18HKACbWPn&lang=en-US&region=US&includeAdjustedClose=true&interval=1d&period1=1514903400&period2=1610085600&events=div%7Csplit&corsDomain=finance.yahoo.com
AKICU has results
trying url: https://query2.finance.yahoo.com/v8/finance/chart/AKRO?formatted=true&crumb=T18HKACbWPn&lang=en-US&region=US&includeAdjustedClose=true&interval=1d&period1=1608906600&period2=1610085600&events=div%7Csplit&corsDomain=finance.yahoo.com
AKRO has results
trying url: https://query2.finance.yahoo.com/v8/finance/chart/AKUS?formatted=true&crumb=T18HKACbWPn&lang=en-US&region=US&includeAdjustedClose=true&interval=1d&period1=1608906600&period2=1610085600&events=div%7Cs

trying url: https://query2.finance.yahoo.com/v8/finance/chart/APM?formatted=true&crumb=T18HKACbWPn&lang=en-US&region=US&includeAdjustedClose=true&interval=1d&period1=1608906600&period2=1610085600&events=div%7Csplit&corsDomain=finance.yahoo.com
APM has results
trying url: https://query2.finance.yahoo.com/v8/finance/chart/APRE?formatted=true&crumb=T18HKACbWPn&lang=en-US&region=US&includeAdjustedClose=true&interval=1d&period1=1608906600&period2=1610085600&events=div%7Csplit&corsDomain=finance.yahoo.com
APRE has results
trying url: https://query2.finance.yahoo.com/v8/finance/chart/APSGU?formatted=true&crumb=T18HKACbWPn&lang=en-US&region=US&includeAdjustedClose=true&interval=1d&period1=1514903400&period2=1610085600&events=div%7Csplit&corsDomain=finance.yahoo.com
trying url: https://query2.finance.yahoo.com/v8/finance/chart/APTX?formatted=true&crumb=T18HKACbWPn&lang=en-US&region=US&includeAdjustedClose=true&interval=1d&period1=1608906600&period2=1610085600&events=div%7Csplit&corsDomain=finan

trying url: https://query2.finance.yahoo.com/v8/finance/chart/AVO?formatted=true&crumb=T18HKACbWPn&lang=en-US&region=US&includeAdjustedClose=true&interval=1d&period1=1608906600&period2=1610085600&events=div%7Csplit&corsDomain=finance.yahoo.com
AVO has results
trying url: https://query2.finance.yahoo.com/v8/finance/chart/AVRO?formatted=true&crumb=T18HKACbWPn&lang=en-US&region=US&includeAdjustedClose=true&interval=1d&period1=1608906600&period2=1610085600&events=div%7Csplit&corsDomain=finance.yahoo.com
AVRO has results
trying url: https://query2.finance.yahoo.com/v8/finance/chart/AVTR?formatted=true&crumb=T18HKACbWPn&lang=en-US&region=US&includeAdjustedClose=true&interval=1d&period1=1608906600&period2=1610085600&events=div%7Csplit&corsDomain=finance.yahoo.com
AVTR has results
trying url: https://query2.finance.yahoo.com/v8/finance/chart/AXLA?formatted=true&crumb=T18HKACbWPn&lang=en-US&region=US&includeAdjustedClose=true&interval=1d&period1=1608906600&period2=1610085600&events=div%7Csplit&

trying url: https://query2.finance.yahoo.com/v8/finance/chart/BLTSU?formatted=true&crumb=T18HKACbWPn&lang=en-US&region=US&includeAdjustedClose=true&interval=1d&period1=1514903400&period2=1610085600&events=div%7Csplit&corsDomain=finance.yahoo.com
BLTSU has results
trying url: https://query2.finance.yahoo.com/v8/finance/chart/BLUWU?formatted=true&crumb=T18HKACbWPn&lang=en-US&region=US&includeAdjustedClose=true&interval=1d&period1=1608906600&period2=1610085600&events=div%7Csplit&corsDomain=finance.yahoo.com
BLUWU has results
trying url: https://query2.finance.yahoo.com/v8/finance/chart/BMRGU?formatted=true&crumb=T18HKACbWPn&lang=en-US&region=US&includeAdjustedClose=true&interval=1d&period1=1514903400&period2=1610085600&events=div%7Csplit&corsDomain=finance.yahoo.com
trying url: https://query2.finance.yahoo.com/v8/finance/chart/BNGOU?formatted=true&crumb=T18HKACbWPn&lang=en-US&region=US&includeAdjustedClose=true&interval=1d&period1=1514903400&period2=1610085600&events=div%7Csplit&corsDomai

...

trying url: https://query2.finance.yahoo.com/v8/finance/chart/SUMO?formatted=true&crumb=T18HKACbWPn&lang=en-US&region=US&includeAdjustedClose=true&interval=1d&period1=1608906600&period2=1610085600&events=div%7Csplit&corsDomain=finance.yahoo.com
SUMO has results
trying url: https://query2.finance.yahoo.com/v8/finance/chart/SURF?formatted=true&crumb=T18HKACbWPn&lang=en-US&region=US&includeAdjustedClose=true&interval=1d&period1=1608906600&period2=1610085600&events=div%7Csplit&corsDomain=finance.yahoo.com
SURF has results
trying url: https://query2.finance.yahoo.com/v8/finance/chart/SVACU?formatted=true&crumb=T18HKACbWPn&lang=en-US&region=US&includeAdjustedClose=true&interval=1d&period1=1608906600&period2=1610085600&events=div%7Csplit&corsDomain=finance.yahoo.com
SVACU has results
trying url: https://query2.finance.yahoo.com/v8/finance/chart/SVMK?formatted=true&crumb=T18HKACbWPn&lang=en-US&region=US&includeAdjustedClose=true&interval=1d&period1=1608906600&period2=1610085600&events=div%7Csp

trying url: https://query2.finance.yahoo.com/v8/finance/chart/TIGR?formatted=true&crumb=T18HKACbWPn&lang=en-US&region=US&includeAdjustedClose=true&interval=1d&period1=1608906600&period2=1610085600&events=div%7Csplit&corsDomain=finance.yahoo.com
TIGR has results
trying url: https://query2.finance.yahoo.com/v8/finance/chart/TINVU?formatted=true&crumb=T18HKACbWPn&lang=en-US&region=US&includeAdjustedClose=true&interval=1d&period1=1514903400&period2=1610085600&events=div%7Csplit&corsDomain=finance.yahoo.com
TINVU NO RESULTS
trying url: https://query2.finance.yahoo.com/v8/finance/chart/TKKSU?formatted=true&crumb=T18HKACbWPn&lang=en-US&region=US&includeAdjustedClose=true&interval=1d&period1=1514903400&period2=1610085600&events=div%7Csplit&corsDomain=finance.yahoo.com
TKKSU NO RESULTS
trying url: https://query2.finance.yahoo.com/v8/finance/chart/TLC?formatted=true&crumb=T18HKACbWPn&lang=en-US&region=US&includeAdjustedClose=true&interval=1d&period1=1608906600&period2=1610085600&events=div%7Cspl

trying url: https://query2.finance.yahoo.com/v8/finance/chart/UBER?formatted=true&crumb=T18HKACbWPn&lang=en-US&region=US&includeAdjustedClose=true&interval=1d&period1=1608906600&period2=1610085600&events=div%7Csplit&corsDomain=finance.yahoo.com
UBER has results
trying url: https://query2.finance.yahoo.com/v8/finance/chart/UBX?formatted=true&crumb=T18HKACbWPn&lang=en-US&region=US&includeAdjustedClose=true&interval=1d&period1=1608906600&period2=1610085600&events=div%7Csplit&corsDomain=finance.yahoo.com
UBX has results
trying url: https://query2.finance.yahoo.com/v8/finance/chart/UCL?formatted=true&crumb=T18HKACbWPn&lang=en-US&region=US&includeAdjustedClose=true&interval=1d&period1=1608906600&period2=1610085600&events=div%7Csplit&corsDomain=finance.yahoo.com
UCL has results
trying url: https://query2.finance.yahoo.com/v8/finance/chart/UMRX?formatted=true&crumb=T18HKACbWPn&lang=en-US&region=US&includeAdjustedClose=true&interval=1d&period1=1608906600&period2=1610085600&events=div%7Csplit&co

trying url: https://query2.finance.yahoo.com/v8/finance/chart/VRM?formatted=true&crumb=T18HKACbWPn&lang=en-US&region=US&includeAdjustedClose=true&interval=1d&period1=1608906600&period2=1610085600&events=div%7Csplit&corsDomain=finance.yahoo.com
VRM has results
trying url: https://query2.finance.yahoo.com/v8/finance/chart/VSPRU?formatted=true&crumb=T18HKACbWPn&lang=en-US&region=US&includeAdjustedClose=true&interval=1d&period1=1608906600&period2=1610085600&events=div%7Csplit&corsDomain=finance.yahoo.com
VSPRU has results
trying url: https://query2.finance.yahoo.com/v8/finance/chart/VSTA?formatted=true&crumb=T18HKACbWPn&lang=en-US&region=US&includeAdjustedClose=true&interval=1d&period1=1608906600&period2=1610085600&events=div%7Csplit&corsDomain=finance.yahoo.com
VSTA has results
trying url: https://query2.finance.yahoo.com/v8/finance/chart/VTAQU?formatted=true&crumb=T18HKACbWPn&lang=en-US&region=US&includeAdjustedClose=true&interval=1d&period1=1608919197&period2=1610085600&events=div%7Cspl

trying url: https://query2.finance.yahoo.com/v8/finance/chart/YJ?formatted=true&crumb=T18HKACbWPn&lang=en-US&region=US&includeAdjustedClose=true&interval=1d&period1=1608906600&period2=1610085600&events=div%7Csplit&corsDomain=finance.yahoo.com
YJ has results
trying url: https://query2.finance.yahoo.com/v8/finance/chart/YMAB?formatted=true&crumb=T18HKACbWPn&lang=en-US&region=US&includeAdjustedClose=true&interval=1d&period1=1608906600&period2=1610085600&events=div%7Csplit&corsDomain=finance.yahoo.com
YMAB has results
trying url: https://query2.finance.yahoo.com/v8/finance/chart/YQ?formatted=true&crumb=T18HKACbWPn&lang=en-US&region=US&includeAdjustedClose=true&interval=1d&period1=1608906600&period2=1610085600&events=div%7Csplit&corsDomain=finance.yahoo.com
YQ has results
trying url: https://query2.finance.yahoo.com/v8/finance/chart/YSACU?formatted=true&crumb=T18HKACbWPn&lang=en-US&region=US&includeAdjustedClose=true&interval=1d&period1=1608906600&period2=1610085600&events=div%7Csplit&corsD

In [18]:
performance_df = pd.concat(performance_df_list)
performance_df

,symbol,unix_time,date,open,close,high,low,volume,date_pulled
0,AAAU,1609165800,2020-12-28,18.799999,18.660000,18.879999,18.639999,487900.0,2021-01-08
1,AAAU,1609252200,2020-12-29,18.700001,18.709999,18.780001,18.650000,432600.0,2021-01-08
2,AAAU,1609338600,2020-12-30,18.740000,18.870001,18.870001,18.730000,400200.0,2021-01-08
3,AAAU,1609425000,2020-12-31,18.889999,18.940001,18.940001,18.830000,282500.0,2021-01-08
4,AAAU,1609770600,2021-01-04,19.330000,19.350000,19.360001,19.219999,767800.0,2021-01-08
...,...,...,...,...,...,...,...,...,...
3,ZUO,1609425000,2020-12-31,14.110000,13.930000,14.120000,13.705000,1354700.0,2021-01-08
4,ZUO,1609770600,2021-01-04,14.000000,13.550000,14.050000,13.080000,2317500.0,2021-01-08
5,ZUO,1609857000,2021-01-05,13.510000,13.400000,13.626000,13.360000,1161100.0,2021-01-08
6,ZUO,1609943400,2021-01-06,13.280000,13.420000,13.710000,13.169000,1419600.0,2021-01-08


In [22]:
# add performance to database
connection = engine.connect()
performance_df.to_sql('performance', con=engine, if_exists='append', index=False)

In [ ]:
# pull for one stock example
# r = requests.get(f'https://query2.finance.yahoo.com/v8/finance/chart/RSVAU?formatted=true&crumb=T18HKACbWPn&lang=en-US&region=US&includeAdjustedClose=true&interval=1d&period1=1514903400&period2=1608357600&events=div%7Csplit&corsDomain=finance.yahoo.com')
# data = r.json()
# print(r)

# timestamp = data["chart"]["result"][0]["timestamp"]
# stk_open = data["chart"]["result"][0]["indicators"]["quote"][0]["open"]
# stk_close = data["chart"]["result"][0]["indicators"]["quote"][0]["close"]
# stk_high = data["chart"]["result"][0]["indicators"]["quote"][0]["high"]
# stk_low = data["chart"]["result"][0]["indicators"]["quote"][0]["low"]
# stk_vol = data["chart"]["result"][0]["indicators"]["quote"][0]["volume"]

# # dataframe with new stock info
# df = pd.DataFrame({"symbol" : "RSVAU",
#                    "unix_time" : timestamp,
#                    "date" : [datetime.fromtimestamp(ts).strftime('%Y-%m-%d') for ts in timestamp],
#                    "open" : stk_open, 
#                    "close" : stk_close,
#                    "high" : stk_high,
#                    "low" : stk_low,
#                    "volume" : stk_vol
#                   })

# df['date'] = pd.to_datetime(df['date'], format="%Y-%m-%d")
# df

# Industry Performance

In [58]:
# get unix time range for web request
start_unixtime = 1514903400  #Jan 2, 2018

current_date = datetime.now()
print(current_date)

earliest_date_time = current_date.replace(hour=0, minute=0, second=0, microsecond=0)
print(earliest_date_time)

end_unixtime = time.mktime(earliest_date_time.timetuple())
end_unixtime = int(end_unixtime)
end_unixtime

2021-01-13 12:33:38.352326
2021-01-13 00:00:00


1610517600

In [59]:
today = datetime.today().strftime("%Y-%m-%d")
print(today)

2021-01-13


In [60]:
# industry index's list for requests
ind_idx_lookup_list = ['%5EDJI', '%5EIXIC', '%5EGSPC']
ind_idx_symbol_list = ['DJI', 'IXIC', 'GSPC']

In [61]:
# initial industry dataframe to interate through for requests
ind_df = pd.DataFrame({'symbol_lookup': ind_idx_lookup_list,
                      'symbol': ind_idx_symbol_list})
ind_df["default_start_unixtime"] = start_unixtime
ind_df["end_unixtime"] = end_unixtime
ind_df

,symbol_lookup,symbol,default_start_unixtime,end_unixtime
0,%5EDJI,DJI,1514903400,1610517600
1,%5EIXIC,IXIC,1514903400,1610517600
2,%5EGSPC,GSPC,1514903400,1610517600


In [62]:
# update to only pull new date information
ind_perf_start_unix = pd.read_sql("SELECT symbol, max(unix_time) AS max_unix_captured  FROM industry_performance  GROUP BY symbol""", connection)
ind_df = ind_df.merge(ind_perf_start_unix, on="symbol", how="outer")
ind_df["max_unix_captured"] = ind_df["max_unix_captured"].fillna(0).astype('int64')
ind_df["max_unix_captured"] = ind_df["max_unix_captured"] + 86400  #add a day to latest date captured
ind_df

,symbol_lookup,symbol,default_start_unixtime,end_unixtime,max_unix_captured
0,%5EDJI,DJI,1514903400,1610517600,1610202600
1,%5EIXIC,IXIC,1514903400,1610517600,1610202600
2,%5EGSPC,GSPC,1514903400,1610517600,1610202600


In [63]:
ind_df["start_unixtime"] = ind_df[["default_start_unixtime", "max_unix_captured"]].max(axis=1).astype('int64') 
ind_df

,symbol_lookup,symbol,default_start_unixtime,end_unixtime,max_unix_captured,start_unixtime
0,%5EDJI,DJI,1514903400,1610517600,1610202600,1610202600
1,%5EIXIC,IXIC,1514903400,1610517600,1610202600,1610202600
2,%5EGSPC,GSPC,1514903400,1610517600,1610202600,1610202600


In [8]:
# empty list of dfs
performance_df_list= []

# make requests for performance information for each row
for row in ind_df.itertuples():

    url = f'https://query2.finance.yahoo.com/v8/finance/chart/{row.symbol_lookup}?formatted=true&crumb=T18HKACbWPn&lang=en-US&region=US&includeAdjustedClose=true&interval=1d&period1={row.start_unixtime}&period2={row.end_unixtime}&events=div%7Csplit&corsDomain=finance.yahoo.com'
    r = requests.get(url)
    print(f"trying url: {url}")
    if r.ok:
        try: 
            data = r.json()

            # get data
            timestamp = data["chart"]["result"][0]["timestamp"]
            stk_open = data["chart"]["result"][0]["indicators"]["quote"][0]["open"]
            stk_close = data["chart"]["result"][0]["indicators"]["quote"][0]["close"]
            stk_high = data["chart"]["result"][0]["indicators"]["quote"][0]["high"]
            stk_low = data["chart"]["result"][0]["indicators"]["quote"][0]["low"]
            stk_vol = data["chart"]["result"][0]["indicators"]["quote"][0]["volume"]

            #transform into dataframe
            df = pd.DataFrame({"symbol" : row.symbol,
                               "unix_time" : timestamp,
                               "date" : [datetime.fromtimestamp(ts).strftime('%Y-%m-%d') for ts in timestamp],
                               "open" : stk_open, 
                               "close" : stk_close,
                               "high" : stk_high,
                               "low" : stk_low,
                               "volume" : stk_vol
                              })

            df['date'] = pd.to_datetime(df['date'], format="%Y-%m-%d")
            df['date_pulled'] = today

            print(f"{row.symbol} has results")
            performance_df_list.append(df)
        except KeyError:
            continue
    
    else:
        print(f"{row.symbol} NO RESULTS")

    time.sleep(.5)
    

trying url: https://query2.finance.yahoo.com/v8/finance/chart/%5EDJI?formatted=true&crumb=T18HKACbWPn&lang=en-US&region=US&includeAdjustedClose=true&interval=1d&period1=1514903400&period2=1610258400&events=div%7Csplit&corsDomain=finance.yahoo.com
DJI has results
trying url: https://query2.finance.yahoo.com/v8/finance/chart/%5EIXIC?formatted=true&crumb=T18HKACbWPn&lang=en-US&region=US&includeAdjustedClose=true&interval=1d&period1=1514903400&period2=1610258400&events=div%7Csplit&corsDomain=finance.yahoo.com
IXIC has results
trying url: https://query2.finance.yahoo.com/v8/finance/chart/%5EGSPC?formatted=true&crumb=T18HKACbWPn&lang=en-US&region=US&includeAdjustedClose=true&interval=1d&period1=1514903400&period2=1610258400&events=div%7Csplit&corsDomain=finance.yahoo.com
GSPC has results


In [9]:
ind_performance_df = pd.concat(performance_df_list)
ind_performance_df

,symbol,unix_time,date,open,close,high,low,volume,date_pulled
0,DJI,1514903400,2018-01-02,24809.349609,24824.009766,24864.189453,24741.699219,341130000,2021-01-10
1,DJI,1514989800,2018-01-03,24850.449219,24922.679688,24941.919922,24825.550781,456790000,2021-01-10
2,DJI,1515076200,2018-01-04,24964.859375,25075.130859,25105.960938,24963.269531,403280000,2021-01-10
3,DJI,1515162600,2018-01-05,25114.919922,25295.869141,25299.789062,25112.009766,358020000,2021-01-10
4,DJI,1515421800,2018-01-08,25308.400391,25283.000000,25311.990234,25235.410156,341390000,2021-01-10
...,...,...,...,...,...,...,...,...,...
756,GSPC,1609770600,2021-01-04,3764.610107,3700.649902,3769.989990,3662.709961,5006680000,2021-01-10
757,GSPC,1609857000,2021-01-05,3698.020020,3726.860107,3737.830078,3695.070068,4582620000,2021-01-10
758,GSPC,1609943400,2021-01-06,3712.199951,3748.139893,3783.040039,3705.340088,6049970000,2021-01-10
759,GSPC,1610029800,2021-01-07,3764.709961,3803.790039,3811.550049,3764.709961,5080870000,2021-01-10


In [10]:
# add performance to database
connection = engine.connect()
ind_performance_df.to_sql('industry_performance', con=engine, if_exists='append', index=False)

# Company Characteristics
This will provide city, industry, etc.  
https://query1.finance.yahoo.com/v10/finance/quoteSummary/AAPL?modules=assetProfile%2CearningsHistory

In [24]:
# get symbols from SQL
connection = engine.connect()
symbols_df = pd.read_sql("SELECT symbol FROM stocks", connection)
symbols_df.head()

,symbol
0,AAAU
1,AACQU
2,ABCL
3,ABCM
4,ABNB


In [25]:
# get symbols that already have company info
symb_in_compinfo = pd.read_sql("SELECT symbol FROM company_info", connection)
symb_in_compinfo.head()

,symbol
0,AAAU
1,ABCL
2,ABCM
3,ABNB
4,ACACU


In [26]:
# remove any symbols company info already avaialable for, only keeping "new" symbols
symbols_df = symbols_df[~symbols_df["symbol"].isin(symb_in_compinfo["symbol"])]
symbols_df

,symbol
1,AACQU
10,ACICU
12,ACNDU
14,ACTTU
17,ADOCU
...,...
965,WOOF
967,WPF'U
973,XPOAU
974,XSPL


In [27]:
#get characteristics dataframe
today = datetime.today().strftime("%Y-%m-%d")

# empty list for df
symbol = []
address = []
city = []
state = []
zip_code = []
country = []
website = []
industry = []
sector = []
business_summary = []

# make requests for characteristic information for each row
for row in symbols_df.itertuples():

    url = f'https://query1.finance.yahoo.com/v10/finance/quoteSummary/{row.symbol}?modules=assetProfile%2CearningsHistory'
    r = requests.get(url)
    print(f"trying url: {url}")
    if r.ok:
        try: 
            data = r.json()
            char_dict = data["quoteSummary"]["result"][0]["assetProfile"]

            symbol.append(row.symbol)
            address.append(char_dict.get("address1", ""))
            city.append(char_dict.get("city", ""))
            state.append(char_dict.get("state", ""))  
            zip_code.append(char_dict.get("zip", ""))  
            country.append(char_dict.get("country", ""))  
            website.append(char_dict.get("website", ""))
            industry.append(char_dict.get("industry", ""))  
            sector.append(char_dict.get("sector", ""))  
            
            # shorten the character length of the full business summary
            bus_summary = char_dict.get("longBusinessSummary", "")
            if len(bus_summary) > 252:
                bus_summary = bus_summary[:995] + '...'
            business_summary.append(bus_summary)
                

        except KeyError:
            continue
    
    else:
        print(f"{row.symbol} NO RESULTS")

    time.sleep(.25)

#transform into dataframe
comp_info_df = pd.DataFrame({"symbol": symbol,
                   "address": address,
                   "city": city,
                   "state": state,
                   "zip_code": zip_code,
                   "country": country,
                   "website": website,
                   "industry": industry,
                   "sector": sector,
                   "business_summary": business_summary})
comp_info_df['date_pulled'] = today
comp_info_df.head()


trying url: https://query1.finance.yahoo.com/v10/finance/quoteSummary/AACQU?modules=assetProfile%2CearningsHistory
AACQU NO RESULTS
trying url: https://query1.finance.yahoo.com/v10/finance/quoteSummary/ACICU?modules=assetProfile%2CearningsHistory
ACICU NO RESULTS
trying url: https://query1.finance.yahoo.com/v10/finance/quoteSummary/ACNDU?modules=assetProfile%2CearningsHistory
ACNDU NO RESULTS
trying url: https://query1.finance.yahoo.com/v10/finance/quoteSummary/ACTTU?modules=assetProfile%2CearningsHistory
trying url: https://query1.finance.yahoo.com/v10/finance/quoteSummary/ADOCU?modules=assetProfile%2CearningsHistory
ADOCU NO RESULTS
trying url: https://query1.finance.yahoo.com/v10/finance/quoteSummary/AFRM?modules=assetProfile%2CearningsHistory
AFRM NO RESULTS
trying url: https://query1.finance.yahoo.com/v10/finance/quoteSummary/AGCB?modules=assetProfile%2CearningsHistory
trying url: https://query1.finance.yahoo.com/v10/finance/quoteSummary/AJAXU?modules=assetProfile%2CearningsHistor

trying url: https://query1.finance.yahoo.com/v10/finance/quoteSummary/DMYTU?modules=assetProfile%2CearningsHistory
DMYTU NO RESULTS
trying url: https://query1.finance.yahoo.com/v10/finance/quoteSummary/DNJR?modules=assetProfile%2CearningsHistory
DNJR NO RESULTS
trying url: https://query1.finance.yahoo.com/v10/finance/quoteSummary/DPHCU?modules=assetProfile%2CearningsHistory
DPHCU NO RESULTS
trying url: https://query1.finance.yahoo.com/v10/finance/quoteSummary/DRVN?modules=assetProfile%2CearningsHistory
DRVN NO RESULTS
trying url: https://query1.finance.yahoo.com/v10/finance/quoteSummary/DWINU?modules=assetProfile%2CearningsHistory
DWINU NO RESULTS
trying url: https://query1.finance.yahoo.com/v10/finance/quoteSummary/EAGLU?modules=assetProfile%2CearningsHistory
EAGLU NO RESULTS
trying url: https://query1.finance.yahoo.com/v10/finance/quoteSummary/EMPWU?modules=assetProfile%2CearningsHistory
EMPWU NO RESULTS
trying url: https://query1.finance.yahoo.com/v10/finance/quoteSummary/ENPCU?modu

trying url: https://query1.finance.yahoo.com/v10/finance/quoteSummary/IPV'U?modules=assetProfile%2CearningsHistory
IPV'U NO RESULTS
trying url: https://query1.finance.yahoo.com/v10/finance/quoteSummary/IVANU?modules=assetProfile%2CearningsHistory
IVANU NO RESULTS
trying url: https://query1.finance.yahoo.com/v10/finance/quoteSummary/JFKKU?modules=assetProfile%2CearningsHistory
JFKKU NO RESULTS
trying url: https://query1.finance.yahoo.com/v10/finance/quoteSummary/JIH'U?modules=assetProfile%2CearningsHistory
JIH'U NO RESULTS
trying url: https://query1.finance.yahoo.com/v10/finance/quoteSummary/JWS'U?modules=assetProfile%2CearningsHistory
JWS'U NO RESULTS
trying url: https://query1.finance.yahoo.com/v10/finance/quoteSummary/KAIRU?modules=assetProfile%2CearningsHistory
trying url: https://query1.finance.yahoo.com/v10/finance/quoteSummary/KCACU?modules=assetProfile%2CearningsHistory
KCACU NO RESULTS
trying url: https://query1.finance.yahoo.com/v10/finance/quoteSummary/KLAQU?modules=assetProf

trying url: https://query1.finance.yahoo.com/v10/finance/quoteSummary/PVT'U?modules=assetProfile%2CearningsHistory
PVT'U NO RESULTS
trying url: https://query1.finance.yahoo.com/v10/finance/quoteSummary/PVTL?modules=assetProfile%2CearningsHistory
PVTL NO RESULTS
trying url: https://query1.finance.yahoo.com/v10/finance/quoteSummary/QES?modules=assetProfile%2CearningsHistory
trying url: https://query1.finance.yahoo.com/v10/finance/quoteSummary/QFIN?modules=assetProfile%2CearningsHistory
trying url: https://query1.finance.yahoo.com/v10/finance/quoteSummary/QLI?modules=assetProfile%2CearningsHistory
trying url: https://query1.finance.yahoo.com/v10/finance/quoteSummary/RBACU?modules=assetProfile%2CearningsHistory
RBACU NO RESULTS
trying url: https://query1.finance.yahoo.com/v10/finance/quoteSummary/RICEU?modules=assetProfile%2CearningsHistory
RICEU NO RESULTS
trying url: https://query1.finance.yahoo.com/v10/finance/quoteSummary/RMG'U?modules=assetProfile%2CearningsHistory
RMG'U NO RESULTS
tr

,symbol,address,city,state,zip_code,country,website,industry,sector,business_summary,date_pulled
0,AGCB,2550 Sand Hill Road,Menlo Park,CA,94025,United States,,Shell Companies,Financial Services,Altimeter Growth Corp. 2 does not have signifi...,2021-01-09
1,AKICU,9705 Collins Avenue 1901 North,Bal Harbour,FL,33154,United States,,,,,2021-01-09
2,BCYPU,300 West 41st Street,Miami Beach,FL,33140,United States,,,,,2021-01-09
3,BENEU,777 SW 37th Avenue,Miami,FL,33135-3250,United States,,Shell Companies,Financial Services,Benessere Capital Acquisition Corp. does not h...,2021-01-09
4,BLTSU,12100 Wilshire Blvd,Los Angeles,CA,90025,United States,,Shell Companies,Financial Services,Bright Lights Acquisition Corp. is a blank che...,2021-01-09


In [28]:
# add company info to database
connection = engine.connect()
comp_info_df.to_sql('company_info', con=engine, if_exists='append', index=False)

In [ ]:
# get single characteristics for testing
# symbol = "VTAQU"
# url = f'https://query1.finance.yahoo.com/v10/finance/quoteSummary/{symbol}?modules=assetProfile%2CearningsHistory'
# r = requests.get(url)
# data = r.json()
# char_dict = data["quoteSummary"]["result"][0]["assetProfile"]

# address = char_dict.get("address1", "")
# city = char_dict.get("city", "")
# state = char_dict.get("state", "")  
# zip_code = char_dict.get("zip", "")  
# country = char_dict.get("country", "")  
# website = char_dict.get("website", "")
# industry = char_dict.get("industry", "")  
# sector = char_dict.get("sector", "")  
# business_summary = char_dict.get("longBusinessSummary", "")  

# characteristics_df = pd.DataFrame({"symbol": symbol,
#                                    "address": address,
#                                    "city": city,
#                                    "state": state,
#                                    "zip_code": zip_code,
#                                    "country": country,
#                                    "website": website,
#                                    "industry": industry,
#                                    "sector": sector,
#                                    "business_summary": business_summary
#                                   }
#                                   ,index=[0]
#                                   )
# characteristics_df

# Market Cap

In [47]:
# get unix time range for web request
start_unixtime = 1514903400  #Jan 2, 2018

current_date = datetime.now()
print(current_date)

earliest_date_time = current_date.replace(hour=0, minute=0, second=0, microsecond=0)
print(earliest_date_time)

end_unixtime = time.mktime(earliest_date_time.timetuple())
end_unixtime = int(end_unixtime)
end_unixtime

2021-01-13 10:07:25.072168
2021-01-13 00:00:00


1610517600

In [48]:
# query stocks from SQL
connection = engine.connect()
ipo_stocks = pd.read_sql("SELECT symbol, deal_status FROM stocks WHERE deal_status = 'priced'", connection)
ipo_stocks["default_start_unixtime"] = start_unixtime
ipo_stocks["end_unixtime"] = end_unixtime
ipo_stocks

,symbol,deal_status,default_start_unixtime,end_unixtime
0,AAAU,priced,1514903400,1610517600
1,AACQU,priced,1514903400,1610517600
2,ABCL,priced,1514903400,1610517600
3,ABCM,priced,1514903400,1610517600
4,ABNB,priced,1514903400,1610517600
...,...,...,...,...
967,ZM,priced,1514903400,1610517600
968,ZNTEU,priced,1514903400,1610517600
969,ZNTL,priced,1514903400,1610517600
970,ZS,priced,1514903400,1610517600


In [49]:
# update to unix to avoid capturing duplicate records
comp_fin_start_unix = pd.read_sql("SELECT symbol, max(unix_timestamp) AS max_unix_captured  FROM market_cap  GROUP BY symbol""", connection)
ipo_stocks = ipo_stocks.merge(comp_fin_start_unix, on="symbol", how="outer")
ipo_stocks["max_unix_captured"] = ipo_stocks["max_unix_captured"].fillna(0).astype('int64')
ipo_stocks["max_unix_captured"] = ipo_stocks["max_unix_captured"] + 604800  #add a week to latest date captured
ipo_stocks

,symbol,deal_status,default_start_unixtime,end_unixtime,max_unix_captured
0,AAAU,priced,1514903400,1610517600,604800
1,AACQU,priced,1514903400,1610517600,604800
2,ABCL,priced,1514903400,1610517600,1610928000
3,ABCM,priced,1514903400,1610517600,1609977600
4,ABNB,priced,1514903400,1610517600,1610928000
...,...,...,...,...,...
967,ZM,priced,1514903400,1610517600,1610668800
968,ZNTEU,priced,1514903400,1610517600,604800
969,ZNTL,priced,1514903400,1610517600,1610668800
970,ZS,priced,1514903400,1610517600,1610928000


In [50]:
ipo_stocks["start_unixtime"] = ipo_stocks[["default_start_unixtime", "max_unix_captured"]].max(axis=1).astype('int64') 
ipo_stocks

,symbol,deal_status,default_start_unixtime,end_unixtime,max_unix_captured,start_unixtime
0,AAAU,priced,1514903400,1610517600,604800,1514903400
1,AACQU,priced,1514903400,1610517600,604800,1514903400
2,ABCL,priced,1514903400,1610517600,1610928000,1610928000
3,ABCM,priced,1514903400,1610517600,1609977600,1609977600
4,ABNB,priced,1514903400,1610517600,1610928000,1610928000
...,...,...,...,...,...,...
967,ZM,priced,1514903400,1610517600,1610668800,1610668800
968,ZNTEU,priced,1514903400,1610517600,604800,1514903400
969,ZNTL,priced,1514903400,1610517600,1610668800,1610668800
970,ZS,priced,1514903400,1610517600,1610928000,1610928000


In [51]:
def fn_get_fin_metric_index(df, metric):
    """returns the index for a given metric is in the list of metrics on yahoo (cannot use a key as it is not a dictionary).
    For example, the metric parameteter may be 'trailingMarketCap'. Other options include:
    
    trailingMarketCap, trailingForwardPeRatio, trailingPeRatio, trailingEnterpriseValue, trailingPegRatio, 
    trailingEnterprisesValueRevenueRatio, trailingEnterprisesValueEBITDARatio, trailingPsRatio, trailingPbRatio
    """
    idx = df[df["metric"] == metric].index
    idx = idx[0]
    return idx

In [52]:
# rolling 12 fields are trailingMarketCap, trailingForwardPeRatio, trailingPeRatio, trailingEnterpriseValue, trailingPegRatio, 
# trailingEnterprisesValueRevenueRatio, trailingEnterprisesValueEBITDARatio, trailingPsRatio, trailingPbRatio

In [53]:
#get market cap dataframe
today = datetime.today().strftime("%Y-%m-%d")

# empty list for df
symbol_list = []
unix_timestamp_list = []
date_list = []
market_cap_list = []
market_cap_fmt_list = []

# make requests for characteristic information for each row
for row in ipo_stocks.itertuples():

    #NOTE NOTE NOTE: WILL NEED UP UPDATE START UNIXTIME AFTER INITIAL RUN
    url = f'https://query1.finance.yahoo.com/ws/fundamentals-timeseries/v1/finance/timeseries/{row.symbol}?lang=en-US&region=US&symbol={row.symbol}&padTimeSeries=true&type=quarterlyMarketCap%2CtrailingMarketCap%2CquarterlyEnterpriseValue%2CtrailingEnterpriseValue%2CquarterlyPeRatio%2CtrailingPeRatio%2CquarterlyForwardPeRatio%2CtrailingForwardPeRatio%2CquarterlyPegRatio%2CtrailingPegRatio%2CquarterlyPsRatio%2CtrailingPsRatio%2CquarterlyPbRatio%2CtrailingPbRatio%2CquarterlyEnterprisesValueRevenueRatio%2CtrailingEnterprisesValueRevenueRatio%2CquarterlyEnterprisesValueEBITDARatio%2CtrailingEnterprisesValueEBITDARatio&merge=false&period1={row.start_unixtime}&period2={row.end_unixtime}&corsDomain=finance.yahoo.com'
    r = requests.get(url)
    
    if r.ok:
        time.sleep(.25)
        data = r.json()
        
        payload = data["timeseries"]["result"]
        index_list = []
        metric_list = []
        count = 0

        for d in range(0, len(payload)):
            metric = payload[d]["meta"]["type"][0] 
            metric_list.append(metric) 
            index_list.append(count)
            count += 1
            
#         print(f"{index_list} : {metric_list}")
        comp_fin_df = pd.DataFrame({"metric": metric_list},
                          index=index_list)
        

        print(f"trying url: {url}")
        try: 
            for n in range(0, len(payload[fn_get_fin_metric_index(comp_fin_df, "trailingMarketCap")]["trailingMarketCap"])):
                symbol_list.append(row.symbol)


                # get the market cap index using the fn_get_fin_metric_index function
                market_cap = payload[fn_get_fin_metric_index(comp_fin_df, "trailingMarketCap")]["trailingMarketCap"]

                unix_timestamp_list.append(payload[fn_get_fin_metric_index(comp_fin_df, "trailingMarketCap")]["timestamp"][n])
                date_list.append(market_cap[n]["asOfDate"])
                market_cap_list.append(market_cap[n]["reportedValue"]["raw"])
                market_cap_fmt_list.append(market_cap[n]["reportedValue"]["fmt"])
        except KeyError:
            continue

trying url: https://query1.finance.yahoo.com/ws/fundamentals-timeseries/v1/finance/timeseries/AAAU?lang=en-US&region=US&symbol=AAAU&padTimeSeries=true&type=quarterlyMarketCap%2CtrailingMarketCap%2CquarterlyEnterpriseValue%2CtrailingEnterpriseValue%2CquarterlyPeRatio%2CtrailingPeRatio%2CquarterlyForwardPeRatio%2CtrailingForwardPeRatio%2CquarterlyPegRatio%2CtrailingPegRatio%2CquarterlyPsRatio%2CtrailingPsRatio%2CquarterlyPbRatio%2CtrailingPbRatio%2CquarterlyEnterprisesValueRevenueRatio%2CtrailingEnterprisesValueRevenueRatio%2CquarterlyEnterprisesValueEBITDARatio%2CtrailingEnterprisesValueEBITDARatio&merge=false&period1=1514903400&period2=1610517600&corsDomain=finance.yahoo.com
trying url: https://query1.finance.yahoo.com/ws/fundamentals-timeseries/v1/finance/timeseries/AACQU?lang=en-US&region=US&symbol=AACQU&padTimeSeries=true&type=quarterlyMarketCap%2CtrailingMarketCap%2CquarterlyEnterpriseValue%2CtrailingEnterpriseValue%2CquarterlyPeRatio%2CtrailingPeRatio%2CquarterlyForwardPeRatio%2Ct

trying url: https://query1.finance.yahoo.com/ws/fundamentals-timeseries/v1/finance/timeseries/ACNDU?lang=en-US&region=US&symbol=ACNDU&padTimeSeries=true&type=quarterlyMarketCap%2CtrailingMarketCap%2CquarterlyEnterpriseValue%2CtrailingEnterpriseValue%2CquarterlyPeRatio%2CtrailingPeRatio%2CquarterlyForwardPeRatio%2CtrailingForwardPeRatio%2CquarterlyPegRatio%2CtrailingPegRatio%2CquarterlyPsRatio%2CtrailingPsRatio%2CquarterlyPbRatio%2CtrailingPbRatio%2CquarterlyEnterprisesValueRevenueRatio%2CtrailingEnterprisesValueRevenueRatio%2CquarterlyEnterprisesValueEBITDARatio%2CtrailingEnterprisesValueEBITDARatio&merge=false&period1=1514903400&period2=1610517600&corsDomain=finance.yahoo.com
trying url: https://query1.finance.yahoo.com/ws/fundamentals-timeseries/v1/finance/timeseries/ACTCU?lang=en-US&region=US&symbol=ACTCU&padTimeSeries=true&type=quarterlyMarketCap%2CtrailingMarketCap%2CquarterlyEnterpriseValue%2CtrailingEnterpriseValue%2CquarterlyPeRatio%2CtrailingPeRatio%2CquarterlyForwardPeRatio%2

trying url: https://query1.finance.yahoo.com/ws/fundamentals-timeseries/v1/finance/timeseries/AGCB?lang=en-US&region=US&symbol=AGCB&padTimeSeries=true&type=quarterlyMarketCap%2CtrailingMarketCap%2CquarterlyEnterpriseValue%2CtrailingEnterpriseValue%2CquarterlyPeRatio%2CtrailingPeRatio%2CquarterlyForwardPeRatio%2CtrailingForwardPeRatio%2CquarterlyPegRatio%2CtrailingPegRatio%2CquarterlyPsRatio%2CtrailingPsRatio%2CquarterlyPbRatio%2CtrailingPbRatio%2CquarterlyEnterprisesValueRevenueRatio%2CtrailingEnterprisesValueRevenueRatio%2CquarterlyEnterprisesValueEBITDARatio%2CtrailingEnterprisesValueEBITDARatio&merge=false&period1=1514903400&period2=1610517600&corsDomain=finance.yahoo.com
trying url: https://query1.finance.yahoo.com/ws/fundamentals-timeseries/v1/finance/timeseries/AGCUU?lang=en-US&region=US&symbol=AGCUU&padTimeSeries=true&type=quarterlyMarketCap%2CtrailingMarketCap%2CquarterlyEnterpriseValue%2CtrailingEnterpriseValue%2CquarterlyPeRatio%2CtrailingPeRatio%2CquarterlyForwardPeRatio%2Ct

trying url: https://query1.finance.yahoo.com/ws/fundamentals-timeseries/v1/finance/timeseries/ALACU?lang=en-US&region=US&symbol=ALACU&padTimeSeries=true&type=quarterlyMarketCap%2CtrailingMarketCap%2CquarterlyEnterpriseValue%2CtrailingEnterpriseValue%2CquarterlyPeRatio%2CtrailingPeRatio%2CquarterlyForwardPeRatio%2CtrailingForwardPeRatio%2CquarterlyPegRatio%2CtrailingPegRatio%2CquarterlyPsRatio%2CtrailingPsRatio%2CquarterlyPbRatio%2CtrailingPbRatio%2CquarterlyEnterprisesValueRevenueRatio%2CtrailingEnterprisesValueRevenueRatio%2CquarterlyEnterprisesValueEBITDARatio%2CtrailingEnterprisesValueEBITDARatio&merge=false&period1=1514903400&period2=1610517600&corsDomain=finance.yahoo.com
trying url: https://query1.finance.yahoo.com/ws/fundamentals-timeseries/v1/finance/timeseries/ALEC?lang=en-US&region=US&symbol=ALEC&padTimeSeries=true&type=quarterlyMarketCap%2CtrailingMarketCap%2CquarterlyEnterpriseValue%2CtrailingEnterpriseValue%2CquarterlyPeRatio%2CtrailingPeRatio%2CquarterlyForwardPeRatio%2Ct

trying url: https://query1.finance.yahoo.com/ws/fundamentals-timeseries/v1/finance/timeseries/AMBO?lang=en-US&region=US&symbol=AMBO&padTimeSeries=true&type=quarterlyMarketCap%2CtrailingMarketCap%2CquarterlyEnterpriseValue%2CtrailingEnterpriseValue%2CquarterlyPeRatio%2CtrailingPeRatio%2CquarterlyForwardPeRatio%2CtrailingForwardPeRatio%2CquarterlyPegRatio%2CtrailingPegRatio%2CquarterlyPsRatio%2CtrailingPsRatio%2CquarterlyPbRatio%2CtrailingPbRatio%2CquarterlyEnterprisesValueRevenueRatio%2CtrailingEnterprisesValueRevenueRatio%2CquarterlyEnterprisesValueEBITDARatio%2CtrailingEnterprisesValueEBITDARatio&merge=false&period1=1610668800&period2=1610517600&corsDomain=finance.yahoo.com
trying url: https://query1.finance.yahoo.com/ws/fundamentals-timeseries/v1/finance/timeseries/AMCIU?lang=en-US&region=US&symbol=AMCIU&padTimeSeries=true&type=quarterlyMarketCap%2CtrailingMarketCap%2CquarterlyEnterpriseValue%2CtrailingEnterpriseValue%2CquarterlyPeRatio%2CtrailingPeRatio%2CquarterlyForwardPeRatio%2Ct

...

trying url: https://query1.finance.yahoo.com/ws/fundamentals-timeseries/v1/finance/timeseries/VGACU?lang=en-US&region=US&symbol=VGACU&padTimeSeries=true&type=quarterlyMarketCap%2CtrailingMarketCap%2CquarterlyEnterpriseValue%2CtrailingEnterpriseValue%2CquarterlyPeRatio%2CtrailingPeRatio%2CquarterlyForwardPeRatio%2CtrailingForwardPeRatio%2CquarterlyPegRatio%2CtrailingPegRatio%2CquarterlyPsRatio%2CtrailingPsRatio%2CquarterlyPbRatio%2CtrailingPbRatio%2CquarterlyEnterprisesValueRevenueRatio%2CtrailingEnterprisesValueRevenueRatio%2CquarterlyEnterprisesValueEBITDARatio%2CtrailingEnterprisesValueEBITDARatio&merge=false&period1=1514903400&period2=1610517600&corsDomain=finance.yahoo.com
trying url: https://query1.finance.yahoo.com/ws/fundamentals-timeseries/v1/finance/timeseries/VHAQU?lang=en-US&region=US&symbol=VHAQU&padTimeSeries=true&type=quarterlyMarketCap%2CtrailingMarketCap%2CquarterlyEnterpriseValue%2CtrailingEnterpriseValue%2CquarterlyPeRatio%2CtrailingPeRatio%2CquarterlyForwardPeRatio%2

trying url: https://query1.finance.yahoo.com/ws/fundamentals-timeseries/v1/finance/timeseries/VMACU?lang=en-US&region=US&symbol=VMACU&padTimeSeries=true&type=quarterlyMarketCap%2CtrailingMarketCap%2CquarterlyEnterpriseValue%2CtrailingEnterpriseValue%2CquarterlyPeRatio%2CtrailingPeRatio%2CquarterlyForwardPeRatio%2CtrailingForwardPeRatio%2CquarterlyPegRatio%2CtrailingPegRatio%2CquarterlyPsRatio%2CtrailingPsRatio%2CquarterlyPbRatio%2CtrailingPbRatio%2CquarterlyEnterprisesValueRevenueRatio%2CtrailingEnterprisesValueRevenueRatio%2CquarterlyEnterprisesValueEBITDARatio%2CtrailingEnterprisesValueEBITDARatio&merge=false&period1=1514903400&period2=1610517600&corsDomain=finance.yahoo.com
trying url: https://query1.finance.yahoo.com/ws/fundamentals-timeseries/v1/finance/timeseries/VMAR?lang=en-US&region=US&symbol=VMAR&padTimeSeries=true&type=quarterlyMarketCap%2CtrailingMarketCap%2CquarterlyEnterpriseValue%2CtrailingEnterpriseValue%2CquarterlyPeRatio%2CtrailingPeRatio%2CquarterlyForwardPeRatio%2Ct

trying url: https://query1.finance.yahoo.com/ws/fundamentals-timeseries/v1/finance/timeseries/WEI?lang=en-US&region=US&symbol=WEI&padTimeSeries=true&type=quarterlyMarketCap%2CtrailingMarketCap%2CquarterlyEnterpriseValue%2CtrailingEnterpriseValue%2CquarterlyPeRatio%2CtrailingPeRatio%2CquarterlyForwardPeRatio%2CtrailingForwardPeRatio%2CquarterlyPegRatio%2CtrailingPegRatio%2CquarterlyPsRatio%2CtrailingPsRatio%2CquarterlyPbRatio%2CtrailingPbRatio%2CquarterlyEnterprisesValueRevenueRatio%2CtrailingEnterprisesValueRevenueRatio%2CquarterlyEnterprisesValueEBITDARatio%2CtrailingEnterprisesValueEBITDARatio&merge=false&period1=1610668800&period2=1610517600&corsDomain=finance.yahoo.com
trying url: https://query1.finance.yahoo.com/ws/fundamentals-timeseries/v1/finance/timeseries/WHD?lang=en-US&region=US&symbol=WHD&padTimeSeries=true&type=quarterlyMarketCap%2CtrailingMarketCap%2CquarterlyEnterpriseValue%2CtrailingEnterpriseValue%2CquarterlyPeRatio%2CtrailingPeRatio%2CquarterlyForwardPeRatio%2Ctrailin

trying url: https://query1.finance.yahoo.com/ws/fundamentals-timeseries/v1/finance/timeseries/XP?lang=en-US&region=US&symbol=XP&padTimeSeries=true&type=quarterlyMarketCap%2CtrailingMarketCap%2CquarterlyEnterpriseValue%2CtrailingEnterpriseValue%2CquarterlyPeRatio%2CtrailingPeRatio%2CquarterlyForwardPeRatio%2CtrailingForwardPeRatio%2CquarterlyPegRatio%2CtrailingPegRatio%2CquarterlyPsRatio%2CtrailingPsRatio%2CquarterlyPbRatio%2CtrailingPbRatio%2CquarterlyEnterprisesValueRevenueRatio%2CtrailingEnterprisesValueRevenueRatio%2CquarterlyEnterprisesValueEBITDARatio%2CtrailingEnterprisesValueEBITDARatio&merge=false&period1=1610668800&period2=1610517600&corsDomain=finance.yahoo.com
trying url: https://query1.finance.yahoo.com/ws/fundamentals-timeseries/v1/finance/timeseries/XPEV?lang=en-US&region=US&symbol=XPEV&padTimeSeries=true&type=quarterlyMarketCap%2CtrailingMarketCap%2CquarterlyEnterpriseValue%2CtrailingEnterpriseValue%2CquarterlyPeRatio%2CtrailingPeRatio%2CquarterlyForwardPeRatio%2Ctrailin

trying url: https://query1.finance.yahoo.com/ws/fundamentals-timeseries/v1/finance/timeseries/YMAB?lang=en-US&region=US&symbol=YMAB&padTimeSeries=true&type=quarterlyMarketCap%2CtrailingMarketCap%2CquarterlyEnterpriseValue%2CtrailingEnterpriseValue%2CquarterlyPeRatio%2CtrailingPeRatio%2CquarterlyForwardPeRatio%2CtrailingForwardPeRatio%2CquarterlyPegRatio%2CtrailingPegRatio%2CquarterlyPsRatio%2CtrailingPsRatio%2CquarterlyPbRatio%2CtrailingPbRatio%2CquarterlyEnterprisesValueRevenueRatio%2CtrailingEnterprisesValueRevenueRatio%2CquarterlyEnterprisesValueEBITDARatio%2CtrailingEnterprisesValueEBITDARatio&merge=false&period1=1610668800&period2=1610517600&corsDomain=finance.yahoo.com
trying url: https://query1.finance.yahoo.com/ws/fundamentals-timeseries/v1/finance/timeseries/YQ?lang=en-US&region=US&symbol=YQ&padTimeSeries=true&type=quarterlyMarketCap%2CtrailingMarketCap%2CquarterlyEnterpriseValue%2CtrailingEnterpriseValue%2CquarterlyPeRatio%2CtrailingPeRatio%2CquarterlyForwardPeRatio%2Ctrailin

In [54]:
#transform into dataframe
market_cap_df = pd.DataFrame({"symbol": symbol_list,
                                   "unix_timestamp": unix_timestamp_list,
                                   "date": date_list,
                                   "market_cap": market_cap_list,
                                   "market_cap_formatted": market_cap_fmt_list, 
                                   "date_pulled": today
                                  })

market_cap_df["market_cap"] = market_cap_df["market_cap"].astype('int64')
market_cap_df

,symbol,unix_timestamp,date,market_cap,market_cap_formatted,date_pulled


In [46]:
# add company info to database
connection = engine.connect()
market_cap_df.to_sql('market_cap', con=engine, if_exists='append', index=False)

In [ ]:
# get single characteristics for testing
# symbol = "AAPL"

# symbol_list = []
# unix_timestamp_list = []
# date_list = []
# market_cap_list = []
# market_cap_fmt_list = []


# url = f'https://query1.finance.yahoo.com/ws/fundamentals-timeseries/v1/finance/timeseries/{symbol}?lang=en-US&region=US&symbol=AAPL&padTimeSeries=true&type=quarterlyMarketCap%2CtrailingMarketCap%2CquarterlyEnterpriseValue%2CtrailingEnterpriseValue%2CquarterlyPeRatio%2CtrailingPeRatio%2CquarterlyForwardPeRatio%2CtrailingForwardPeRatio%2CquarterlyPegRatio%2CtrailingPegRatio%2CquarterlyPsRatio%2CtrailingPsRatio%2CquarterlyPbRatio%2CtrailingPbRatio%2CquarterlyEnterprisesValueRevenueRatio%2CtrailingEnterprisesValueRevenueRatio%2CquarterlyEnterprisesValueEBITDARatio%2CtrailingEnterprisesValueEBITDARatio&merge=false&period1=1514903400&period2=1608962400&corsDomain=finance.yahoo.com'
# r = requests.get(url)
# data = r.json()

# for n in range(0, len(payload[fn_get_fin_metric_index(comp_fin_df, "trailingMarketCap")]["trailingMarketCap"])):
#     symbol_list.append(symbol)
    
    
#     # get the market cap index using the fn_get_fin_metric_index function
#     market_cap = payload[fn_get_fin_metric_index(comp_fin_df, "trailingMarketCap")]["trailingMarketCap"]
    
#     unix_timestamp_list.append(payload[fn_get_fin_metric_index(comp_fin_df, "trailingMarketCap")]["timestamp"][n])
#     date_list.append(market_cap[n]["asOfDate"])
#     period_type_list.append(market_cap[n]["periodType"])
#     market_cap_list.append(market_cap[n]["reportedValue"]["raw"])
#     market_cap_fmt_list.append(market_cap[n]["reportedValue"]["fmt"])
    
# comp_market_cap_df = pd.DataFrame({"symbol": symbol_list,
#                                    "unix_timestamp": unix_timestamp_list,
#                                    "date": date_list,
#                                    "market_cap": market_cap_list,
#                                    "market_cap_formatted": market_cap_fmt_list, 
#                                    "date_pulled": today
#                                   })

# comp_market_cap_df["market_cap"] = comp_market_cap_df["market_cap"].astype('int64')
# comp_market_cap_df

# Analysis

Information that may be interesting to share.  Examples include:
1. Timing of when it launches, how long its been, etc.
2. Price performance
    - Launch date open and close price (how they did on first day)
    - How did it do when its hit 1 month, 3 month, 6 month, 1 year milestone
3. Industry perormance
    - Did it outperform the S&P 
    - Did it outperform they sector (Ex: tech, consumer goods)
4. Top performers
    - Which IPOs did best in last 1 month, 3 month, 6 month, 1 year milestone

In [ ]:
# Print text for each record

for i in range(0,len(response)):
  print(f"{response[i]['expected_to_trade']}: {response[i]['company']} [{response[i]['symbol_proposed']}]. Price (Low-High): ${response[i]['price_low']}-{response[i]['price_high']}. #new_ipo_{response[i]['symbol_proposed']}")  

In [ ]:
# Establish Twitter connection

import tweepy

CONSUMER_KEY = "consumer_key"
CONSUMER_SECRET = "consumer_secret"   
ACCESS_KEY = "access_key"    
ACCESS_SECRET = "access_secret"

auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
auth.set_access_token(ACCESS_KEY, ACCESS_SECRET)

api = tweepy.API(auth)

In [ ]:
# Post a tweet for each record
for i in range(0,len(response)):
  new_tweet = f"{response[i]['expected_to_trade']}: {response[i]['company']} [{response[i]['symbol_proposed']}]. Price (Low-High): ${response[i]['price_low']}-{response[i]['price_high']}. #new_ipo_{response[i]['symbol_proposed']}" 
  api.update_status(new_tweet)      